In [ ]:
# Install required packages.
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

1.12.1+cu113
     |████████████████████████████████| 7.9 MB 14.4 MB/s 
     |████████████████████████████████| 3.5 MB 8.6 MB/s 


In [ ]:
import torch
from torch_geometric.datasets import Planetoid
from torch_geometric.transforms import NormalizeFeatures

dataset = Planetoid(root='data/Planetoid', name='PubMed', transform=NormalizeFeatures())

print()
print(f'Dataset: {dataset}:')
print('==================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')

data = dataset[0]  # Get the first graph object.

print()
print(data)
print('===============================================================================================================')

# Gather some statistics about the graph.
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
print(f'Number of training nodes: {data.train_mask.sum()}')
print(f'Training node label rate: {int(data.train_mask.sum()) / data.num_nodes:.3f}')
print(f'Has isolated nodes: {data.has_isolated_nodes()}')
print(f'Has self-loops: {data.has_self_loops()}')
print(f'Is undirected: {data.is_undirected()}')

Processing...



Dataset: PubMed():
Number of graphs: 1
Number of features: 500
Number of classes: 3

Data(x=[19717, 500], edge_index=[2, 88648], y=[19717], train_mask=[19717], val_mask=[19717], test_mask=[19717])
Number of nodes: 19717
Number of edges: 88648
Average node degree: 4.50
Number of training nodes: 60
Training node label rate: 0.003
Has isolated nodes: False
Has self-loops: False
Is undirected: True


Done!


As can be seen, this graph has around 19,717 nodes.
While this number of nodes should fit into GPU memory with ease, it's nonetheless a good example to showcase how one can scale GNNs up within PyTorch Geometric.

**Cluster-GCN** ([Chiang et al. (2019)](https://arxiv.org/abs/1905.07953) works by first partioning the graph into subgraphs based on graph partitioning algorithms.
With this, GNNs are restricted to solely convolve inside their specific subgraphs, which omits the problem of **neighborhood explosion**.

However, after the graph is partitioned, some links are removed which may limit the model's performance due to a biased estimation.
To address this issue, Cluster-GCN also **incorporates between-cluster links inside a mini-batch**, which results in the following **stochastic partitioning scheme**:

Here, colors represent the adjacency information that is maintained per batch (which is potentially different for every epoch).

PyTorch Geometric provides a **two-stage implementation** of the Cluster-GCN algorithm:
1. [**`ClusterData`**](https://pytorch-geometric.readthedocs.io/en/latest/modules/data.html#torch_geometric.data.ClusterData) converts a `Data` object into a dataset of subgraphs containing `num_parts` partitions.
2. Given a user-defined `batch_size`, [**`ClusterLoader`**](https://pytorch-geometric.readthedocs.io/en/latest/modules/data.html#torch_geometric.data.ClusterLoader) implements the stochastic partitioning scheme in order to create mini-batches.

The procedure to craft mini-batches then looks as follows:

In [ ]:
from torch_geometric.loader import ClusterData, ClusterLoader

torch.manual_seed(12345)
cluster_data = ClusterData(data, num_parts = 128) # 1. Create subgraphs
train_loader = ClusterLoader(cluster_data, batch_size = 32, shuffle = True)

print()
total_num_nodes = 0
for step, sub_data in enumerate(train_loader):
  print(f'Step {step + 1}:')
  print('=======')
  print(f'Number of nodes in the current batch: {sub_data.num_nodes}')
  print(sub_data)
  print()
  total_num_nodes += sub_data.num_nodes

print(f'Iterated over {total_num_nodes} of {data.num_nodes} nodes!')

Computing METIS partitioning...



Step 1:
Number of nodes in the current batch: 4928
Data(x=[4928, 500], y=[4928], train_mask=[4928], val_mask=[4928], test_mask=[4928], edge_index=[2, 16174])

Step 2:
Number of nodes in the current batch: 4937
Data(x=[4937, 500], y=[4937], train_mask=[4937], val_mask=[4937], test_mask=[4937], edge_index=[2, 17832])

Step 3:
Number of nodes in the current batch: 4927
Data(x=[4927, 500], y=[4927], train_mask=[4927], val_mask=[4927], test_mask=[4927], edge_index=[2, 14712])

Step 4:
Number of nodes in the current batch: 4925
Data(x=[4925, 500], y=[4925], train_mask=[4925], val_mask=[4925], test_mask=[4925], edge_index=[2, 18006])

Iterated over 19717 of 19717 nodes!


Done!


Here, we partition the initial graph into **128 partitions**, and use a **`batch_size` of 32 subgraphs** to form mini-batches (leaving us with 4 batches per epoch).
As one can see, after a single epoch, each node has been seen exactly once.

The great thing about Cluster-GCN is that it does not complicate the GNN model implementation.
Here, we can make use of the **exactly same architecture** introduced in [the second chapter of this tutorial](https://colab.research.google.com/drive/14OvFnAXggxB8vM4e8vSURUp1TaKnovzX).

In [ ]:
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GCN, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GCNConv(dataset.num_node_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, dataset.num_classes)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)
        return x

model = GCN(hidden_channels=16)
print(model)

GCN(
  (conv1): GCNConv(500, 16)
  (conv2): GCNConv(16, 3)
)


In [ ]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))

model = GCN(hidden_channels=16)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
criterion = torch.nn.CrossEntropyLoss()

def train():
      model.train()

      for sub_data in train_loader:  # Iterate over each mini-batch.
          out = model(sub_data.x, sub_data.edge_index)  # Perform a single forward pass.
          loss = criterion(out[sub_data.train_mask], sub_data.y[sub_data.train_mask])  # Compute the loss solely based on the training nodes.
          loss.backward()  # Derive gradients.
          optimizer.step()  # Update parameters based on gradients.
          optimizer.zero_grad()  # Clear gradients.

def test():
      model.eval()
      out = model(data.x, data.edge_index)
      pred = out.argmax(dim=1)  # Use the class with highest probability.
      
      accs = []
      for mask in [data.train_mask, data.val_mask, data.test_mask]:
          correct = pred[mask] == data.y[mask]  # Check against ground-truth labels.
          accs.append(int(correct.sum()) / int(mask.sum()))  # Derive ratio of correct predictions.
      return accs

for epoch in range(1, 51):
    loss = train()
    train_acc, val_acc, test_acc = test()
    print(f'Epoch: {epoch:03d}, Train: {train_acc:.4f}, Val Acc: {val_acc:.4f}, Test Acc: {test_acc:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train: 0.3333, Val Acc: 0.4160, Test Acc: 0.4070
Epoch: 002, Train: 0.4833, Val Acc: 0.4420, Test Acc: 0.4270
Epoch: 003, Train: 0.7333, Val Acc: 0.5220, Test Acc: 0.5160
Epoch: 004, Train: 0.8333, Val Acc: 0.5920, Test Acc: 0.5930
Epoch: 005, Train: 0.9167, Val Acc: 0.6920, Test Acc: 0.6810
Epoch: 006, Train: 0.9000, Val Acc: 0.6680, Test Acc: 0.6680
Epoch: 007, Train: 0.8833, Val Acc: 0.6520, Test Acc: 0.6530
Epoch: 008, Train: 0.9167, Val Acc: 0.7020, Test Acc: 0.6940
Epoch: 009, Train: 0.9333, Val Acc: 0.7340, Test Acc: 0.7290
Epoch: 010, Train: 0.9333, Val Acc: 0.6880, Test Acc: 0.6900
Epoch: 011, Train: 0.9500, Val Acc: 0.7240, Test Acc: 0.7330
Epoch: 012, Train: 0.9500, Val Acc: 0.7500, Test Acc: 0.7300
Epoch: 013, Train: 0.9667, Val Acc: 0.7680, Test Acc: 0.7530
Epoch: 014, Train: 0.9667, Val Acc: 0.7600, Test Acc: 0.7450
Epoch: 015, Train: 0.9667, Val Acc: 0.7680, Test Acc: 0.7550
Epoch: 016, Train: 0.9667, Val Acc: 0.7660, Test Acc: 0.7630
Epoch: 017, Train: 0.966